In [2]:
#http://paper.people.com.cn/rmrb/page/2018-03/25/01/rmrb2018032501.pdf
import urllib
import datetime
import requests
from bs4 import BeautifulSoup  


from PyPDF2 import PdfFileMerger, PdfFileReader



url = "http://paper.people.com.cn/rmrb/html/2018-03/25/nbs.D110000renmrb_01.htm"
pdf_url01 = "http://paper.people.com.cn/rmrb/page/2018-03/25/01/rmrb2018032501.pdf"
pdf_url02 = "http://paper.people.com.cn/rmrb/page/2018-03/25/02/rmrb2018032502.pdf"


select_day = "2018.04.15"
[year, month, day] = select_day.split(".")
# now = datetime.datetime.now()
# year = str(now.year)
# month = str(now.month).zfill(2)
# day = str(now.day).zfill(2)

# Generate the first page html
paper_people_base = "http://paper.people.com.cn/rmrb/html/"
paper_people_download_base = "http://paper.people.com.cn/rmrb/"

current_time = year + '-' + month + '/' + day + "/nbs.D110000renmrb_01.htm"
url = paper_people_base + current_time
print url

# Generate the pdf list for donwloading


#web_raw = urllib.urlopen(url)
html = requests.get(url) 
html.encoding = 'utf-8' #这一行是将编码转为utf-8否则中文会显示乱码。  

#print type(html)
#print html.text
#print type(web_raw)
#print web_raw.read()




ImportError: No module named PyPDF2

In [4]:
soup = BeautifulSoup(html.text, "html.parser")
#print soup.prettify()



#right_title_pdf_class = soup.find_all('div', class_="right_title-pdf",)
#print right_title_pdf_class
#print type(right_title_pdf_class)

a_href_list = soup.select('a[href^="../../../page"]') # '^' 匹配字符串开头
#print a_href_list 
#print type(a_href_list)
#print len(a_href_list)
#for each_a_href in a_href_list:
#    print each_a_href.get('href')[-38:]
    
download_pdf_list = [each_a_href.get('href')[-37:] for each_a_href in a_href_list]
#print download_pdf_list

download_pdf_list_remove_duplicate = []
for i in download_pdf_list:
  if i not in download_pdf_list_remove_duplicate:
    download_pdf_list_remove_duplicate.append(i)
    
print download_pdf_list_remove_duplicate

for current_pdf in download_pdf_list_remove_duplicate:
    current_file_name = current_pdf.split('/')[-1]
    current_url = paper_people_download_base + current_pdf
    print "Downloading " + current_file_name + " now."
    response = urllib.urlopen(current_url)
    file = open(current_file_name, 'wb')
    file.write(response.read())
    file.close()
    print "Finished."
    print "---------------"

[u'page/2018-04/15/01/rmrb2018041501.pdf', u'page/2018-04/15/02/rmrb2018041502.pdf', u'page/2018-04/15/03/rmrb2018041503.pdf', u'page/2018-04/15/04/rmrb2018041504.pdf', u'page/2018-04/15/05/rmrb2018041505.pdf', u'page/2018-04/15/06/rmrb2018041506.pdf', u'page/2018-04/15/07/rmrb2018041507.pdf', u'page/2018-04/15/08/rmrb2018041508.pdf', u'page/2018-04/15/09/rmrb2018041509.pdf', u'page/2018-04/15/10/rmrb2018041510.pdf', u'page/2018-04/15/11/rmrb2018041511.pdf', u'page/2018-04/15/12/rmrb2018041512.pdf']
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------


In [5]:
#pdf_url01 = "http://paper.people.com.cn/rmrb/page/2018-03/25/01/rmrb2018032501.pdf"

#file_name = pdf_url01.split('/')[-1]
#print (file_name)

#response = urllib.urlopen(pdf_url01)
#file = open("rmrb2018032501.pdf", 'wb')
#file.write(response.read())
#file.close()
#print("Completed")



In [6]:
import os
import sys
import glob
import re 

from PyPDF2 import PdfFileReader,PdfFileWriter

def getFileName(filepath):
    file_list = []
    for root,dirs,files in os.walk(filepath):
        for filespath in files:
            # print(os.path.join(root,filespath))
            file_list.append(os.path.join(root,filespath))

    return file_list

filepath = os.getcwd()
#infnList = ['rmrb2018032501.pdf','rmrb2018032502.pdf']
#print infnList
current_file_name_list = getFileName(filepath)
current_file_name_filter = "rmrb" + year + month + day
infnList = []
for each_file in current_file_name_list:
    each_file_name = each_file.split('/')[-1]
    if each_file.split('/')[-1].startswith(current_file_name_filter):
        infnList.append(each_file_name)

infnList =  sorted(infnList)
print infnList


['rmrb2018041501.pdf', 'rmrb2018041502.pdf', 'rmrb2018041503.pdf', 'rmrb2018041504.pdf', 'rmrb2018041505.pdf', 'rmrb2018041506.pdf', 'rmrb2018041507.pdf', 'rmrb2018041508.pdf', 'rmrb2018041509.pdf', 'rmrb2018041510.pdf', 'rmrb2018041511.pdf', 'rmrb2018041512.pdf']


In [7]:
pdf_output = PdfFileWriter()
for infn in infnList:
    print infn
    pdf_input = PdfFileReader(open(infn, 'rb'), strict=False)
    # 获取 pdf 共用多少页
    page_count = pdf_input.getNumPages()
    print(page_count)
    for i in range(page_count):
        pdf_output.addPage(pdf_input.getPage(i))

summaryName = select_day +'_People_Summary.pdf'
pdf_output.write(open(summaryName, 'wb'))
print("Completed")

rmrb2018041501.pdf
1
rmrb2018041502.pdf
1
rmrb2018041503.pdf
1
rmrb2018041504.pdf
1
rmrb2018041505.pdf
1
rmrb2018041506.pdf
1
rmrb2018041507.pdf
1
rmrb2018041508.pdf
1
rmrb2018041509.pdf
1
rmrb2018041510.pdf
1
rmrb2018041511.pdf
1
rmrb2018041512.pdf
1


Completed
